## Usando arquivo .env para controlar variaveis de ambiente
Para evitar exposição da chave `OPENAI_API_KEY` optei por utilizar arquivo `.env` com a informação da chave.

Para seguir o mesmo método basta criar um arquivo `.env` no mesmo diretório do arquivo `Solucao RAG.ipynb`.
A importação da chave será feita através da célula abaixo que faz a instalação de um biblioteca para carregar
os valores do arquivo `.env`.

In [17]:
%pip install python-dotenv
import dotenv
%load_ext dotenv
%dotenv

Note: you may need to restart the kernel to use updated packages.
The dotenv extension is already loaded. To reload it, use:
  %reload_ext dotenv


In [18]:
from langchain_community.document_loaders.generic import GenericLoader
from langchain_community.document_loaders.parsers import LanguageParser
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings, ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

import os
from git import Repo

In [19]:
repo_path = "./test_repo"

In [20]:
repo = Repo.clone_from("https://github.com/langchain-ai/langchain", to_path=repo_path)

In [23]:
loader = GenericLoader.from_filesystem(
    repo_path + "/libs/core/langchain_core/",
    glob="**/*",
    suffixes=[".py"],
    exclude=["**/non-utf-8-encoding.py"],
    parser=LanguageParser(language=Language.PYTHON, parser_threshold=500)
)

documents = loader.load()
len(documents)

475

In [24]:
python_spliter = RecursiveCharacterTextSplitter.from_language(
    language=Language.PYTHON, chunk_size=2000, chunk_overlap = 200
)

texts = python_spliter.split_documents(documents)
len(texts)

1417

In [25]:
db = Chroma.from_documents(texts, OpenAIEmbeddings(disallowed_special=()))

retriever = db.as_retriever(
    search_type = "mmr",
    search_kwargs = {"k": 8},
)

In [26]:
llm = ChatOpenAI(
    model_name="gpt-3.5-turbo",
    max_tokens=200,
)

In [29]:
prompt = ChatPromptTemplate.from_messages([
    (
        "system",
        "Você é um revisor de código experiente. Forneça informações detalhadas sobre a revisão do código e sugestões de melhorias baseadas no contexto fornecido abaixo: \n\n{context}",
    ),
    ("user", "{input}"),
])

document_chain = create_stuff_documents_chain(llm, prompt)

retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [32]:
response = retrieval_chain.invoke({"input": "Você pode revisar e sugerir melhorias para o código de RunnableBinding"})
response["answer"]

'O código de `RunnableBinding` parece estar bem estruturado e organizado. No entanto, há algumas sugestões de melhorias e observações que podem ser consideradas:\n\n1. **Documentação Adequada**: Adicionar documentação detalhada e explicativa para cada método e classe, incluindo descrições claras, exemplos de uso e possíveis cenários de aplicação.\n\n2. **Tipagem Forte**: Certifique-se de adicionar anotações de tipo explícitas para parâmetros de método e valores de retorno, para facilitar a compreensão do código e garantir a consistência.\n\n3. **Tratamento de Exceções**: Considere adicionar tratamento de exceções apropriado em métodos como `backward`, `update` e outros, para lidar com possíveis erros de forma mais robusta.\n\n4. **Melhorias nos Métodos**: \n   - No'